# Unit Testing `GiRaFFE_NRPy`: Fluxes of $\tilde{S}_i$

### Author: Patrick Nelson

This notebook validates our new, NRPyfied HLLE solver against the function from the original `GiRaFFE` that calculates the flux for $\tilde{S}_i$ according to the the method of Harten, Lax, von Leer, and Einfeldt (HLLE), assuming that we have calculated the values of the flux on the cell faces according to the piecewise-parabolic method (PPM) of [Colella and Woodward (1984)](https://crd.lbl.gov/assets/pubs_presos/AMCS/ANAG/A141984.pdf), modified for the case of GRFFE. 

**Module Status:** <font color=red><b> In-Progress: </b></font> Ideally, later versions will generate the NRPy+ version on the fly and download the ETK version from bitbucket.

**Validation Notes:** Once this is completed, it will show the validation of [Tutorial-GiRaFFE_NRPy_Ccode_library-Stilde-flux](../Tutorial-GiRaFFE_NRPy_Ccode_library-Stilde-flux.ipynb).

It is, in general, good coding practice to unit test functions individually to verify that they produce the expected and intended output. Here, we expect our functions `GRFFE__S_*__flux.C` to produce identical output to the function `GRFFE__S_i__flux.C` in the original `GiRaFFE`. It should be noted that the two codes handle the parameter `flux_dirn` (the direction in which the code is presently calculating the flux through the cell) differently; in the original `GiRaFFE`, the function `GRFFE__S_i__flux()` expects a parameter `flux_dirn` with value 1, 2, or 3, corresponding to the functions `GRFFE__S_0__flux()`, `GRFFE__S_1__flux()`, and `GRFFE__S_2__flux()`, respectively, in `GiRaFFE_NRPy`.

We'll write this in C because the codes we want to test are already written that way, and we would like to avoid modifying the files as much as possible. To do so, we will write the C code as a string literal, and then print it to a file. We will begin by including our new files. 

In [1]:
out_string = """
// The NRPy+ versions of the function. These should require relatively little modification.
// We will need this define, though:
#define REAL double
#include "PPM/GRFFE__S_0__flux.C"
#include "PPM/GRFFE__S_1__flux.C"
#include "PPM/GRFFE__S_2__flux.C"

"""

Next, we'll include the file from the old `GiRaFFE`. But before we can do so, we should define modified versions of the CCTK macros.

In [2]:
out_string += """
#define CCTK_REAL double

"""

We'll also need to download the file in question from the `GiRaFFE` bitbucket repository. This code was originally written by Leo Werneck in the IllinoisGRMHD documentation, modified to download the files we want.

In [3]:
# First download the original IllinoisGRMHD source code
import os,sys
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

import urllib
import os
import cmdline_helper as cmd

out_dir  = "Validation"
cmd.mkdir(out_dir)

original_file_url  = ["https://bitbucket.org/zach_etienne/wvuthorns/raw/5611b2f0b17135538c9d9d17c7da062abe0401b6/GiRaFFE/src/GRFFE__S_i__flux.C",\
                      "https://bitbucket.org/zach_etienne/wvuthorns/raw/5611b2f0b17135538c9d9d17c7da062abe0401b6/GiRaFFE/src/GiRaFFE_headers.h",\
                      "https://bitbucket.org/zach_etienne/wvuthorns/raw/5611b2f0b17135538c9d9d17c7da062abe0401b6/GiRaFFE/src/inlined_functions.C"\
                     ]
original_file_name = ["GRFFE__S_i__flux-original.C",\
                      "GiRaFFE_headers-original.h",\
                      "inlined_functions-original.C"\
                     ]

for i in range(len(original_file_url)):
    original_file_path = os.path.join(out_dir,original_file_name[i])

    # Then download the original IllinoisGRMHD source code
    # We try it here in a couple of ways in an attempt to keep
    # the code more portable
    try:
        original_file_code = urllib.request.urlopen(original_file_url[i]).read()
    except:
        original_file_code = urllib.urlopen(original_file_url[i]).read()

    # Write down the file the original IllinoisGRMHD source code
    with open(original_file_path,"w") as file:
        file.write(original_file_code)
    
    # We add the following lines to append includes to the code we're writing
    out_string += """#include \""""
    out_string += original_file_path
    out_string +=""""
"""

Now we can write a main function. In this function, we will fill all relevant arrays with (appropriate) random values. That is, if a certain gridfunction should never be negative, we will make sure to only generate positive numbers for it.

In [4]:
out_string +="""
// These are common packages that we are likely to need.
#include "stdio.h"
#include "stdlib.h"
#include "math.h"
#include "string.h" // Needed for strncmp, etc.
#include "stdint.h" // Needed for Windows GCC 6.x compatibility

// Let's also #define the NRPy+ gridfunctions
#define ALPHA_FACEGF 0
#define GAMMADET_FACEGF 1
#define GAMMA_FACEDD00GF 2
#define GAMMA_FACEDD01GF 3
#define GAMMA_FACEDD02GF 4
#define GAMMA_FACEDD11GF 5
#define GAMMA_FACEDD12GF 6
#define GAMMA_FACEDD22GF 7
#define GAMMA_FACEUU00GF 8
#define GAMMA_FACEUU11GF 9
#define GAMMA_FACEUU22GF 10
#define BETA_FACEU0GF 11
#define BETA_FACEU1GF 12
#define BETA_FACEU2GF 13
#define VALENCIAV_RU0GF 14
#define VALENCIAV_RU1GF 15
#define VALENCIAV_RU2GF 16
#define B_RU0GF 17
#define B_RU1GF 18
#define B_RU2GF 19
#define VALENCIAV_LU0GF 20
#define VALENCIAV_LU1GF 21
#define VALENCIAV_LU2GF 22
#define B_LU0GF 23
#define B_LU1GF 24
#define B_LU2GF 25
#define U4UPPERZERO_LGF 26
#define U4UPPERZERO_RGF 27
#define NUM_AUXEVOL_GFS 28

int main() {
    // We'll define all indices to be 0. No need to complicate memory access
    i0 = 0;
    i1 = 0;
    i2 = 0;
    
    // This is the array to which we'll write the NRPy+ variables.
    REAL *auxevol_gfs  = (REAL *)malloc(sizeof(REAL) * NUM_AUXEVOL_GFS);
    
    // These are the arrays to which we will write the ETK variables.
    CCTK_REAL METRIC_LAP_PSI4[NUMVARS_METRIC_AUX];
    CCTK_REAL Ur[MAXNUMVARS];
    CCTK_REAL Ul[MAXNUMVARS];
    CCTK_REAL FACEVAL[NUMVARS_FOR_METRIC_FACEVALS];
    CCTK_REAL cmax, CCTK_REAL cmin;
    CCTK_REAL st_x_flux, CCTK_REAL st_y_flux, CCTK_REAL st_z_flux;
    
    // Now, it's time to make the random numbers.
    // We take care to make sure the corresponding quantities have the SAME value.
    auxevol_gfs[IDX4(ALPHA_FACEGF, i0,i1,i2)] = (double)rand()/RAND_MAX*2.0-1.0;
    METRIC_LAP_PSI4[LAPSE] = auxevol_gfs[IDX4(ALPHA_FACEGF, i0,i1,i2)];
    
    METRIC_LAP_PSI4[LAPSE] = 1.0/METRIC_LAP_PSI4[LAPSE];
    auxevol_gfs[IDX4(GAMMADET_FACEGF, i0,i1,i2)];
    METRIC_LAP_PSI4[PSI6] = auxevol_gfs[IDX4(GAMMADET_FACEGF, i0,i1,i2)];
    METRIC_LAP_PSI4[PSI2] = pow(METRIC_LAP_PSI4[PSI6],1.0/3.0);
    METRIC_LAP_PSI4[PSI4] = METRIC_LAP_PSI4[PSI2]*METRIC_LAP_PSI4[PSI2];
    METRIC_LAP_PSI4[PSIM4] = 1.0/METRIC_LAP_PSI4[PSI4];
    
    auxevol_gfs[IDX4(GAMMA_FACEDD00GF, i0,i1,i2)] = (double)rand()/RAND_MAX*2.0-1.0;
    FACEVAL[GXX] = auxevol_gfs[IDX4(GAMMA_FACEDD00GF, i0,i1,i2)];
    auxevol_gfs[IDX4(GAMMA_FACEDD01GF, i0,i1,i2)] = (double)rand()/RAND_MAX*2.0-1.0;
    FACEVAL[GXY] = auxevol_gfs[IDX4(GAMMA_FACEDD01GF, i0,i1,i2)];
    auxevol_gfs[IDX4(GAMMA_FACEDD02GF, i0,i1,i2)] = (double)rand()/RAND_MAX*2.0-1.0;
    FACEVAL[GXZ] = auxevol_gfs[IDX4(GAMMA_FACEDD02GF, i0,i1,i2)];
    auxevol_gfs[IDX4(GAMMA_FACEDD11GF, i0,i1,i2)] = (double)rand()/RAND_MAX*2.0-1.0;
    FACEVAL[GYY] = auxevol_gfs[IDX4(GAMMA_FACEDD11GF, i0,i1,i2)];
    auxevol_gfs[IDX4(GAMMA_FACEDD12GF, i0,i1,i2)] = (double)rand()/RAND_MAX*2.0-1.0;
    FACEVAL[GYZ] = auxevol_gfs[IDX4(GAMMA_FACEDD12GF, i0,i1,i2)];
    auxevol_gfs[IDX4(GAMMA_FACEDD22GF, i0,i1,i2)] = (double)rand()/RAND_MAX*2.0-1.0;
    FACEVAL[GZZ] = auxevol_gfs[IDX4(GAMMA_FACEDD22GF, i0,i1,i2)];
    
    auxevol_gfs[IDX4(GAMMA_FACEUU00GF, i0,i1,i2)] = (double)rand()/RAND_MAX*2.0-1.0;
    FACEVAL[GUPXX] = auxevol_gfs[IDX4(GAMMA_FACEUU00GF, i0,i1,i2)];
    auxevol_gfs[IDX4(GAMMA_FACEUU11GF, i0,i1,i2)] = (double)rand()/RAND_MAX*2.0-1.0;
    FACEVAL[GUPYY] = auxevol_gfs[IDX4(GAMMA_FACEUU11GF, i0,i1,i2)];
    auxevol_gfs[IDX4(GAMMA_FACEUU22GF, i0,i1,i2)] = (double)rand()/RAND_MAX*2.0-1.0;
    FACEVAL[GUPZZ] = auxevol_gfs[IDX4(GAMMA_FACEUU22GF, i0,i1,i2)];
    
    auxevol_gfs[IDX4(BETA_FACEU0GF, i0,i1,i2)] = (double)rand()/RAND_MAX*2.0-1.0;
    FACEVAL[SHIFTX] = auxevol_gfs[IDX4(BETA_FACEU0GF, i0,i1,i2)];
    auxevol_gfs[IDX4(BETA_FACEU1GF, i0,i1,i2)] = (double)rand()/RAND_MAX*2.0-1.0;
    FACEVAL[SHIFTY] = auxevol_gfs[IDX4(BETA_FACEU1GF, i0,i1,i2)];
    auxevol_gfs[IDX4(BETA_FACEU2GF, i0,i1,i2)] = (double)rand()/RAND_MAX*2.0-1.0;
    FACEVAL[SHIFTZ] = auxevol_gfs[IDX4(BETA_FACEU2GF, i0,i1,i2)];
    
    auxevol_gfs[IDX4(VALENCIAV_RU0GF, i0,i1,i2)] = (double)rand()/RAND_MAX*2.0-1.0;
    Ur[VX] = auxevol_gfs[IDX4(VALENCIAV_RU0GF, i0,i1,i2)];
    auxevol_gfs[IDX4(VALENCIAV_RU1GF, i0,i1,i2)] = (double)rand()/RAND_MAX*2.0-1.0;
    Ur[VY] = auxevol_gfs[IDX4(VALENCIAV_RU1GF, i0,i1,i2)];
    auxevol_gfs[IDX4(VALENCIAV_RU2GF, i0,i1,i2)] = (double)rand()/RAND_MAX*2.0-1.0;
    Ur[VZ] = auxevol_gfs[IDX4(VALENCIAV_RU2GF, i0,i1,i2)];
    
    auxevol_gfs[IDX4(B_RU0GF, i0,i1,i2)] = (double)rand()/RAND_MAX*2.0-1.0;
    Ur[BX_CENTER] = auxevol_gfs[IDX4(B_RU0GF, i0,i1,i2)];
    auxevol_gfs[IDX4(B_RU1GF, i0,i1,i2)] = (double)rand()/RAND_MAX*2.0-1.0;
    Ur[BY_CENTER] = auxevol_gfs[IDX4(B_RU1GF, i0,i1,i2)];
    auxevol_gfs[IDX4(B_RU2GF, i0,i1,i2)] = (double)rand()/RAND_MAX*2.0-1.0;
    Ur[BZ_CENTER] = auxevol_gfs[IDX4(B_RU2GF, i0,i1,i2)];
    
    auxevol_gfs[IDX4(VALENCIAV_LU0GF, i0,i1,i2)] = (double)rand()/RAND_MAX*2.0-1.0;
    Ul[VX] = auxevol_gfs[IDX4(VALENCIAV_LU0GF, i0,i1,i2)];
    auxevol_gfs[IDX4(VALENCIAV_LU1GF, i0,i1,i2)] = (double)rand()/RAND_MAX*2.0-1.0;
    Ul[VY] = auxevol_gfs[IDX4(VALENCIAV_LU1GF, i0,i1,i2)];
    auxevol_gfs[IDX4(VALENCIAV_LU2GF, i0,i1,i2)] = (double)rand()/RAND_MAX*2.0-1.0;
    Ul[VZ] = auxevol_gfs[IDX4(VALENCIAV_LU2GF, i0,i1,i2)];
    
    auxevol_gfs[IDX4(B_LU0GF, i0,i1,i2)] = (double)rand()/RAND_MAX*2.0-1.0;
    Ul[BX_CENTER] = auxevol_gfs[IDX4(B_LU0GF, i0,i1,i2)];
    auxevol_gfs[IDX4(B_LU1GF, i0,i1,i2)] = (double)rand()/RAND_MAX*2.0-1.0;
    Ul[BY_CENTER] = auxevol_gfs[IDX4(B_LU1GF, i0,i1,i2)];
    auxevol_gfs[IDX4(B_LU2GF, i0,i1,i2)] = (double)rand()/RAND_MAX*2.0-1.0;
    Ul[BZ_CENTER] = auxevol_gfs[IDX4(B_LU2GF, i0,i1,i2)];
    
    // Now, we'll run the NRPy+ and ETK functions, once in each flux_dirn.
    // We'll compare the output in-between each 
    GRFFE__S_0__flux(0,0,0, auxevol_gfs)
    GRFFE__S_i__flux(0,0,0,0,Ul,Ur,FACEVAL,FACEVAL_LAPSE_PSI4,cmax,cmin,st_x_flux,st_y_flux,st_z_flux);    
    
    GRFFE__S_1__flux(0,0,0, auxevol_gfs)
    GRFFE__S_i__flux(0,0,0,1,Ul,Ur,FACEVAL,FACEVAL_LAPSE_PSI4,cmax,cmin,st_x_flux,st_y_flux,st_z_flux);    

    
    GRFFE__S_2__flux(0,0,0, auxevol_gfs)
    GRFFE__S_i__flux(0,0,0,2,Ul,Ur,FACEVAL,FACEVAL_LAPSE_PSI4,cmax,cmin,st_x_flux,st_y_flux,st_z_flux);    

}
"""